In [1]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import os

# Net Structure
def Net1(inp_np, out_np, epochs, NmU, NmNut):

    # Divide train and test
    inp_np_train = inp_np[::2,0:NmU]
    inp_np_test = inp_np[1::2,0:NmU]
    out_np_train = out_np[::2,0:NmNut]
    out_np_test = out_np[1::2,0:NmNut]
    
    # Create NN Network 1
    Nin = inp_np_train.shape[1]
    Nout = out_np_train.shape[1]
    model = torch.nn.Sequential(
        torch.nn.Linear(Nin, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, Nout),
    )
    
    # Convert to torch
    inp_train = torch.from_numpy(inp_np_train).type(torch.float32)
    out_train = torch.from_numpy(out_np_train).type(torch.float32)
    inp_test = torch.from_numpy(out_np_train).type(torch.float32)
    out_test = torch.from_numpy(out_np_test).type(torch.float32)
    
    # Loss Functions
    loss_fn = torch.nn.MSELoss(reduction='mean')
    
    learning_rate = 1e-4
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    tplot = []
    lossplottrain = []
    lossplottest = []
    
    for t in range(epochs):
        # Forward pass: compute predicted y by passing x to the model.
        y_pred_train = model(inp_train)
        y_pred_test = model(inp_test)
    
        # Compute and print loss.
        loss_train = loss_fn(y_pred_train, out_train)
        loss_test = loss_fn(y_pred_test, out_test)
        if t % 100 == 99:
            print(t, "loss on train" , loss_train.item())
            print(t, "loss on test" , loss_test.item())
            tplot.append(t)
            lossplottrain.append(loss_train.item())
            lossplottest.append(loss_test.item())
    
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()
    
        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss_train.backward()
    
        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
    return tplot,lossplottrain,lossplottest,model

## Input Data
# Number of modes for U
NmU = 10 
# Number of modes for Nut
NmNut = 10
# Number of Epochs
Epochs = 10000

# Read the coefficients
inp_np = np.load("coeffL2U.npy")
out_np = np.load("coeffL2Nut.npy")
# Read Angles from file
angles_train = np.load("angles_train.npy")

# Add angles to train data in a different set
inp_np2 = np.zeros([inp_np.shape[0],inp_np.shape[1]+1])
inp_np2[:,0] = angles_train[:]
inp_np2[:,1::] = inp_np

In [2]:
# Train a net without angles
t_plot1, lossplottrain1, lossplottest1,model1 = Net1(inp_np, out_np, Epochs, NmU, NmNut)

99 loss on train 0.12687833607196808
99 loss on test 0.13562247157096863
199 loss on train 0.017802299931645393
199 loss on test 0.02205485664308071
299 loss on train 0.014593040570616722
299 loss on test 0.023138388991355896
399 loss on train 0.011222034692764282
399 loss on test 0.02675808034837246
499 loss on train 0.008462929166853428
499 loss on test 0.03091924451291561
599 loss on train 0.006792896427214146
599 loss on test 0.03425504267215729
699 loss on train 0.005464176181703806
699 loss on test 0.036693025380373
799 loss on train 0.0043638925999403
799 loss on test 0.039448533207178116
899 loss on train 0.003512861905619502
899 loss on test 0.04181451350450516
999 loss on train 0.002903418382629752
999 loss on test 0.0432993546128273
1099 loss on train 0.0024456819519400597
1099 loss on test 0.04466768354177475
1199 loss on train 0.00208392646163702
1199 loss on test 0.04593091085553169
1299 loss on train 0.0018179899780079722
1299 loss on test 0.04718679189682007
1399 loss o

In [ ]:
# Train a net with angles
t_plot2, lossplottrain2, lossplottest2,model2 = Net1(inp_np2, out_np, Epochs, NmU, NmNut)


In [ ]:
plt.plot(t_plot1, lossplottrain1, label="train1")
plt.plot(t_plot1, lossplottest1, label="test1")
plt.plot(t_plot2, lossplottrain2, label="train2")
plt.plot(t_plot2, lossplottest2, label="test2")
plt.legend()
plt.show()

In [ ]:
# Divide train and test
inp_np_train = inp_np[::2,0:NmU]
inp_np_test = inp_np[1::2,0:NmU]
out_np_train = out_np[::2,0:NmNut]
out_np_test = out_np[1::2,0:NmNut]
int_torch = torch.from_numpy(inp_np_train[100,:]).type(torch.float32)

In [ ]:
out_model = model1(int_torch)

In [ ]:
out_model = model1(torch.from_numpy(inp_np_train[100,:]).type(torch.float32))

In [ ]:
out_torch = torch.from_numpy(out_np_train[100,:]).type(torch.float32)

In [ ]:
out_model - out_torch